In [6]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAI, ChatOpenAI
from dotenv import load_dotenv

In [7]:
load_dotenv()

tools = [TavilySearchResults(max_results=1)] # + load_tools(["openweathermap-api"])

In [8]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)
# llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
# llm = ChatOllama(model="llama3.1", temperature=0)

In [9]:
# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-tools-agent")

# agent = create_openai_tools_agent(llm, tools, prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [10]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
LangChain is a framework designed to facilitate the development of applications powered by large language models (LLMs). It provides a suite of tools and components that help developers build applications that can effectively utilize LLMs for a variety of tasks. LangChain is particularly useful for creating applications that involve:

1. **Data Augmented Generation**: Enhancing the capabilities of language models by integrating them with external data sources.
2. **Agentic Behavior**: Allowing language models to interact with their environment and make decisions based on that interaction.
3. **Chain of Thought Reasoning**: Enabling models to perform complex reasoning by breaking down tasks into a series of steps or "chains."

LangChain offers modular components that can be used to construct these types of applications, making it easier for developers to leverage the power of large language models in a structured and efficient manner.

> Finished 

{'input': 'what is LangChain?',
 'output': 'LangChain is a framework designed to facilitate the development of applications powered by large language models (LLMs). It provides a suite of tools and components that help developers build applications that can effectively utilize LLMs for a variety of tasks. LangChain is particularly useful for creating applications that involve:\n\n1. **Data Augmented Generation**: Enhancing the capabilities of language models by integrating them with external data sources.\n2. **Agentic Behavior**: Allowing language models to interact with their environment and make decisions based on that interaction.\n3. **Chain of Thought Reasoning**: Enabling models to perform complex reasoning by breaking down tasks into a series of steps or "chains."\n\nLangChain offers modular components that can be used to construct these types of applications, making it easier for developers to leverage the power of large language models in a structured and efficient manner.'}

# Adding in Memory

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [12]:
message_history = ChatMessageHistory()

In [13]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [14]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [15]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
You mentioned that your name is Bob.

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'You mentioned that your name is Bob.'}

In [16]:
weather_results = agent_with_chat_history.invoke(
    {"input": "I live in London.  What is the weather like where I live?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)['output']



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'London weather today'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.5171, 'lon': -0.1062, 'tz_id': 'Europe/London', 'localtime_epoch': 1735008778, 'localtime': '2024-12-24 02:52'}, 'current': {'last_updated_epoch': 1735008300, 'last_updated': '2024-12-24 02:45', 'temp_c': 7.1, 'temp_f': 44.7, 'is_day': 0, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/night/248.png', 'code': 1135}, 'wind_mph': 4.0, 'wind_kph': 6.5, 'wind_degree': 246, 'wind_dir': 'WSW', 'pressure_mb': 1024.0, 'pressure_in': 30.23, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 97, 'cloud': 100, 'feelslike_c': 5.9, 'feelslike_f': 42.7, 'windchill_c': 5.9, 'windchill_f': 42.7, 'heatindex_c': 7.1, 'heatindex_f': 44.7, 'dewpoint_c': 6.6, 'dewpoint_f': 44.0, 'vis_km': 0.0, 'vis_m

In [17]:
from openai import OpenAI
from IPython.display import Audio

client = OpenAI()
response = client.audio.speech.create(
    model='tts-1',
    voice='alloy',
    input=weather_results,
)

Audio(response.read())